In [82]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import data_loader
import data_references
from energy_channels import EnergyChannel
import datetime
import matplotlib.pyplot as plt

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import matplotlib.colors as colors
import plot_tools
import numpy as np
import pandas as pd
import psd_tool
from field_models import model
import data_cache
import scipy


import importlib
importlib.reload(plot_tools)
importlib.reload(data_loader)
importlib.reload(data_references)
importlib.reload(psd_tool)
importlib.reload(data_cache)

%matplotlib qt

c:\Dev\Research\REPT_Enhancements_Tool\src\plot_tools.py:55: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("viridis"),


In [83]:
start = datetime.datetime(year = 2013, month = 1, day = 1)
end = datetime.datetime(year = 2013, month = 7, day = 15)

In [84]:
l_cut = 5.0
tol = 0.25

In [85]:
refs_A = data_loader.load_compressed_rept_data(satellite="A", start=start, end=end)

fesa_A = refs_A["FESA"] 
L_A = refs_A["L"] 
mlt_A = refs_A["MLT"]
epoch_A = refs_A["EPOCH"]
energies = refs_A["ENERGIES"]

refs_B = data_loader.load_compressed_rept_data(satellite="B", start=start, end=end)

fesa_B = refs_B["FESA"] 
L_B = refs_B["L"] 
mlt_B = refs_B["MLT"]
epoch_B = refs_B["EPOCH"]
energies = refs_B["ENERGIES"]

satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))

fesa_A = fesa_A[:, (energies < 3.5)]
fesa_B = fesa_B[:, (energies < 3.5)]

fesa_A_DF = pd.DataFrame(fesa_A[satisfies_L_cut_A, :], index=epoch_A[satisfies_L_cut_A])
fesa_B_DF = pd.DataFrame(fesa_B[satisfies_L_cut_B, :], index=epoch_B[satisfies_L_cut_B])

fesa = pd.concat([fesa_A_DF, fesa_B_DF], sort=False).sort_index()
fesa = fesa.resample("5h").mean()

Loading REPT data between: 2013-01-01 00:00:00 and 2013-07-15 00:00:00.
Loading : REPT_201301_A.npz
Loading : REPT_201302_A.npz
Loading : REPT_201303_A.npz
Loading : REPT_201304_A.npz
Loading : REPT_201305_A.npz
Loading : REPT_201306_A.npz
Loading : REPT_201307_A.npz
Loading REPT data between: 2013-01-01 00:00:00 and 2013-07-15 00:00:00.
Loading : REPT_201301_B.npz
Loading : REPT_201302_B.npz
Loading : REPT_201303_B.npz
Loading : REPT_201304_B.npz
Loading : REPT_201305_B.npz
Loading : REPT_201306_B.npz
Loading : REPT_201307_B.npz


In [86]:
field_model = model.TS04D

B_l = 0.311653 / (l_cut ** 3)
E_l = [1.8, 2.1, 2.6, 3.4]
M_e = scipy.constants.physical_constants["electron mass energy equivalent in MeV"][0]

mu_vals = [(E**2 + 2 * M_e * E) / (2 * M_e * B_l) for E in E_l]
#mu_vals = [500, 1000, 3000, 5000, 8000]
chosen_k = 0.07

In [87]:
psd_refs_A = data_loader.load_psd(satellite = "A", field_model = field_model, start = start, end = end)
psd_refs_B = data_loader.load_psd(satellite = "B", field_model = field_model, start = start, end = end)

2013-01-01 00:00:00
Loading : PSD_201301_A_TS04D.npz
2013-02-01 00:00:00
Loading : PSD_201302_A_TS04D.npz
2013-03-01 00:00:00
Loading : PSD_201303_A_TS04D.npz
2013-04-01 00:00:00
Loading : PSD_201304_A_TS04D.npz
2013-05-01 00:00:00
Loading : PSD_201305_A_TS04D.npz
2013-06-01 00:00:00
Loading : PSD_201306_A_TS04D.npz
2013-07-01 00:00:00
Loading : PSD_201307_A_TS04D.npz
2013-01-01 00:00:00
Loading : PSD_201301_B_TS04D.npz
2013-02-01 00:00:00
Loading : PSD_201302_B_TS04D.npz
2013-03-01 00:00:00
Loading : PSD_201303_B_TS04D.npz
2013-04-01 00:00:00
Loading : PSD_201304_B_TS04D.npz
2013-05-01 00:00:00
Loading : PSD_201305_B_TS04D.npz
2013-06-01 00:00:00
Loading : PSD_201306_B_TS04D.npz
2013-07-01 00:00:00
Loading : PSD_201307_B_TS04D.npz


In [88]:
for chosen_mu in mu_vals:
    
    print(f"Selecting PSD for A at mu = {chosen_mu}, k = {chosen_k}")
    
    selected_psd_refs_A = psd_tool.select_mu_and_k_from_psd(refs=psd_refs_A, chosen_mu=chosen_mu, chosen_k=chosen_k)
    
    print(f"Selecting PSD for B at mu = {chosen_mu}, k = {chosen_k}")

    selected_psd_refs_B = psd_tool.select_mu_and_k_from_psd(refs=psd_refs_B, chosen_mu=chosen_mu, chosen_k=chosen_k)

    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs_A, satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs_B, satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)


Selecting PSD for A at mu = 1993.5075698637097, k = 0.07
(93600, 35, 102) (93600,) (93600,) (93600, 102) (93600, 35) (93600, 35) (93600, 35) (93600,)
Time taken for loop: 104.58374214172363
Selecting PSD for B at mu = 1993.5075698637097, k = 0.07
(93120, 35, 102) (93120,) (93120,) (93120, 102) (93120, 35) (93120, 35) (93120, 35) (93120,)
Time taken for loop: 75.28089213371277
Selecting PSD for A at mu = 2573.0048161529007, k = 0.07
(93600, 35, 102) (93600,) (93600,) (93600, 102) (93600, 35) (93600, 35) (93600, 35) (93600,)
Time taken for loop: 114.50060105323792
Selecting PSD for B at mu = 2573.0048161529007, k = 0.07
(93120, 35, 102) (93120,) (93120,) (93120, 102) (93120, 35) (93120, 35) (93120, 35) (93120,)
Time taken for loop: 106.61493802070618
Selecting PSD for A at mu = 3695.815137520753, k = 0.07
(93600, 35, 102) (93600,) (93600,) (93600, 102) (93600, 35) (93600, 35) (93600, 35) (93600,)
Time taken for loop: 108.4760091304779
Selecting PSD for B at mu = 3695.815137520753, k = 0.

In [90]:
fig, axs = plt.subplots(2, 1, sharex=True)


num_energies = fesa.shape[1]
energy_colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=energy_colors(E), linewidth=2.5)
    
axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED, SPIN-AVERAGED")


num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("plasma", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    satisfies_Lstar_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    satisfies_Lstar_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))

    epoch_A_cut = epoch_A[satisfies_Lstar_cut_A]
    PSD_A_cut = PSD_A[satisfies_Lstar_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_Lstar_cut_B]
    PSD_B_cut = PSD_B[satisfies_Lstar_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[1].semilogy(PSD.index, PSD, linewidth=2, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)")
    
    
axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].set_title(rf"L* = {l_cut}, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}, RBSP-COMBINED")
axs[1].set_ylabel("PSD (c/Mev/G)")
axs[1].legend(loc = "upper left")
axs[1].set_xlim(start, end)


C:\Users\brofe\AppData\Local\Temp\ipykernel_26912\4223901689.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  energy_colors = plt.cm.get_cmap("plasma", num_energies)


(15706.0, 15901.0)